<a href="https://colab.research.google.com/github/deondrae4088/WebMd_chat/blob/main/Medai_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Apr  9 13:00:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [3]:
#Importing libraries and dependencies needed for the project
import transformers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

#library not covered in class requirement met
import torch
import pandas as pd

In [4]:
#Found a Pretrained Question and Answer model(meta-llama2-tm) in huggingface
model_name = "deepset/roberta-base-squad2"

tokenizer = transformers.AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = transformers.AutoModelForCausalLM.from_pretrained("deepset/roberta-base-squad2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
#moutn google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#read in the medical condition dataset, questions and answers
medical_df = pd.read_csv('/content/drive/MyDrive/WebMD/medical_qa_data.csv')

In [7]:
medical_df.head()

,Question Type,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."


## BASELINE PRETRAINED MODEL FROM HUGGINFACE

In [8]:
#Test out the roberta-base qa model on simple medical questions

model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

for index, row in medical_df.iterrows():
  context = str(row['Answer'])

  # a) Get predictions
  QA_input = {
    'context': context,
    'question': 'What are the different symptoms of lung cancer'
  }
response = nlp(QA_input)
print(response)

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{'score': 1.0654857760528103e-05, 'start': 302, 'end': 377, 'answer': 'abdominal pain, abdominal mass and symptoms of gastrointestinal obstruction'}


FINETUNE (OPTIMIZE PRETRAINED MODEL)

In [9]:
medical_df_filtered = medical_df[["Question", "Answer"]]

In [10]:
#Preprcoess medical_df_filtered for finetuning

def preprocess_function(examples):
    questions = [q.strip() for q in examples["Question"]]
    inputs = tokenizer(
        questions,
        examples["Answer"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    # max_length = 512,
    #truncation="true"

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["Answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = 0
        end_char = start_char + len(answer)
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [11]:
#split data into training and testing set
from sklearn.model_selection import train_test_split
from datasets import Dataset

train_df, test_df = train_test_split(medical_df_filtered, test_size=0.2)

In [12]:
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

In [13]:
train_df = Dataset.from_pandas(train_df)
test_df = Dataset.from_pandas(test_df)

In [14]:
train_dataset_tokenized = train_df.map(preprocess_function, batched=True, remove_columns=['Question', 'Answer', '__index_level_0__'])
test_dataset_tokenized = test_df.map(preprocess_function, batched=True, remove_columns=['Question', 'Answer', '__index_level_0__'])

Map:   0%|          | 0/13125 [00:00<?, ? examples/s]

Map:   0%|          | 0/3282 [00:00<?, ? examples/s]

In [16]:
#Train the model
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator

model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

data_collator = default_data_collator
training_args = TrainingArguments(
    output_dir="my_medical_qa_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=train_dataset_tokenized,
    processing_class=tokenizer,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.016700,0.008943
2,0.003300,0.001977
3,0.003000,0.001081


TrainOutput(global_step=4923, training_loss=0.07406158128115019, metrics={'train_runtime': 4216.8992, 'train_samples_per_second': 9.337, 'train_steps_per_second': 1.167, 'total_flos': 7716419847360000.0, 'train_loss': 0.07406158128115019, 'epoch': 3.0})

In [17]:
torch.export

<module 'torch.export' from '/usr/local/lib/python3.11/dist-packages/torch/export/__init__.py'>

In [18]:
import os
print(os.listdir("my_medical_qa_model"))

['checkpoint-3500', 'checkpoint-4000', 'checkpoint-2500', 'checkpoint-1500', 'checkpoint-4500', 'checkpoint-2000', 'checkpoint-1000', 'checkpoint-4923', 'runs', 'checkpoint-500', 'checkpoint-3000']


## TEST THE FINETUNED MODEL

In [19]:
from transformers import AutoTokenizer
question = "What are the symptoms of Lymphocytic Choriomeningitis?"
context = str(medical_df["Answer"])
max_length = 512
context = context[:max_length]

tokenizer = AutoTokenizer.from_pretrained("my_medical_qa_model/checkpoint-4923")
inputs = tokenizer(question, context, return_tensors="pt")

In [20]:
model = AutoModelForQuestionAnswering.from_pretrained("my_medical_qa_model/checkpoint-4923")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [21]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
results = tokenizer.decode(predict_answer_tokens)
print(results)

0        LCMV infections can occur after exposure to fr...
1        LCMV is most commonly recognized as causing ne...
2        Individuals of all ages who come into contact ...
3        During the first phase of the disease, the mos...
4        Aseptic meningitis, encephalitis, or meningoen...
                               ...                        
16402    What are the signs and symptoms of Familial vi...
16403    Pseudopelade of Brocq (PBB) is a slowly progre...
16404    What are the signs and symptoms


In [23]:
# prompt: add gradio

!pip install gradio

import gradio as gr
import re

# ... (Your existing code) ...
pd.set_option('display.max_colwidth', 100)

def predict_answer(question):
    context = str(medical_df["Answer"])  # Assuming context remains the same
    tokenizer = AutoTokenizer.from_pretrained("my_medical_qa_model/checkpoint-4923")
    inputs = tokenizer(question, context, return_tensors="pt")
    model = AutoModelForQuestionAnswering.from_pretrained("my_medical_qa_model/checkpoint-4923")
    with torch.no_grad():
        outputs = model(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()
    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    results = tokenizer.decode(predict_answer_tokens)
    match = re.search(r'0\s+(.*?)(?=\s*1\s+)', results, re.DOTALL)
    print(match)
    result = match.group(1).strip()
    print(result)
    #result = " LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  ..."
    return (result)

iface = gr.Interface(
    fn=predict_answer,
    inputs=gr.Textbox(lines=2, placeholder="Enter your medical question here..."),
    outputs=gr.Textbox(lines=10,label="Answer"),
    title="Medical Question Answering",
    description="Ask a question and get an answer from the fine-tuned model."
)

iface.launch(show_error=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2e0fc6782e7e3f4c8c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/sp

## EVALUATION METRICS